In [2]:
from bs4 import BeautifulSoup
import urllib.request
import re
import time
import datetime
from datetime import datetime, timedelta, date
from translate import Translator

**1. Достаём ссылки

In [29]:
top_url = 'https://www.avito.ru/moskva/kommercheskaya_nedvizhimost'

In [7]:
def get_html(url):
    page = urllib.request.urlopen(url)
    html = page.read().decode('UTF-8')
    soup = BeautifulSoup(html, "html.parser")
    return soup

In [20]:
def write_links(divs):
    all_links = open ('.\\All_links.txt', 'a')
    for div in divs:
        link = re.search('(?<=/moskva/kommercheskaya_nedvizhimost)/.*?(?=")', str(div))
        if link!=None:
            all_links.write(link.group(0)+'\n')
    all_links.close()

In [18]:
def get_all_links(top_url):
    url = top_url + '?view=list'
    n=1
    while url != '':
        soup = get_html(url)
        time.sleep(5)
        divs = soup.find_all("div", class_=re.compile("title description-title"))
        write_links(divs)
        if_next = soup.find("a", class_=re.compile("pagination-page js-pagination-next"))
        print(url)
        if if_next==None:
            url=''
        else:
            n+=1
            url = top_url + '?p=' + str(n) + '&view=list'

In [ ]:
get_all_links(top_url)

**2. Обкачиваем ссылки.

В итоге у меня получилось меньше 10000 файлов. Объявления меняются/удаляются с какой-то невероятной скоростью. Вытаскивала информацию на следующий день после получаения ссылок - более 1000 уже были недействительными. Надеюсь, это не повлияет ни на что.

In [46]:
def write_info(url, what):
    path = '.\\All_files\\' + url + '.txt'
    f = open(path, 'a', encoding = 'UTF-8')
    f.write(what + '\n')
    return

In [69]:
def change_date(date):
    now = datetime.now()
    translator= Translator(to_lang="Russian")
    time = re.search('\d*:\d*', date)
    time = time.group(0)
    if "сегодня" in date:
        dt = datetime.strftime(datetime.now(), "%d %B")
    else:
        dt = (datetime.now() - timedelta(1)).strftime('%d %B')
    dt = dt.split(' ')
    day = dt[0]
    month = translator.translate(dt[1])
    date =day + ' ' + month + ' ' +'в' +' ' + time
    return date

In [109]:
def get_title(url, soup):
    title_html = soup.find("span", class_=re.compile("title-info-title-text"))
    title = re.search('(?<="name">).*?(?=</span)', str(title_html))
    title = 'Объявление: ' + title_html.text + '.'
    write_info(url, title)
    return

In [90]:
def get_date(url, soup):
    date_html = soup.find("div", class_=re.compile("title-info-metadata-item"))
    date = re.search ('(?<=размещено ).*?(?=  </div>)', str(date_html))
    if date is None:
        date = 'Дата размещения недоступна, потому что объявление закрыто.'
    else:
        date = date.group(0)
        if "сегодня" or "вчера" in date:
            date = change_date(date)
        date = 'Размещено: ' + date + '.'   
    write_info(url, date)
    return

In [133]:
def get_price(url, soup):
    price_html = soup.find("span", class_=re.compile("price-value-string js-price-value-string"))
    if "в месяц" in str(price_html):
        price =  'Цена: ' + (soup.find("span", class_=re.compile("js-item-price"))).text + ' рублей в месяц.'
    elif "js-item-price" not in str(price_html):
        price = re.sub('\n ', '', price_html.text).strip(' ')+'.'
    else:
        price = 'Цена: ' + (soup.find("span", class_=re.compile("js-item-price"))).text + ' рублей.'
    write_info(url, price)
    return

In [86]:
def get_address(url, soup):
    address_html = soup.find("span", class_=re.compile("item-map-address"))
    if address_html is None:
        address = 'Адрес не указан.'
    else:
        address = 'Адрес: ' + re.sub('\n ', '', address_html.text).strip(' ') + '.'
    write_info(url, address)
    return

In [62]:
def get_text(url, soup):
    descr_html = soup.find("div", class_=re.compile("item-description"))
    descr_p = descr_html.find_all("p")
    write_info(url, 'Текст объявления:')
    for item in descr_p:
        text = re.sub('<.*?>', '', str(item))
        write_info(url, text)
    return

In [93]:
with open('.\\All_links.txt') as f:
    links = f.readlines()

In [ ]:
for i in range(0, len(links)):
    link = links[i].strip('\n')
    url = top_url+link
    soup = get_html(url)
    time.sleep(5)
    if soup.find("span", class_=re.compile("title-info-title-text")) is None:
         continue
    else:
        get_title(link, soup)
        get_date(link, soup)
        get_price(link, soup)
        get_address(link, soup)
        get_text(link, soup)